In [2]:
import os
import time

from dotenv import load_dotenv
from ragie  import Ragie

load_dotenv()

True

In [3]:
ragie = Ragie(
    auth=os.getenv('RAGIE_API_KEY')
)

In [4]:
response = ragie.documents.list()
documents = response.result.documents

for document in documents:
    try:
        ragie.documents.delete(
            document_id=document.id
        )
        print(f"Deleted document {document.id}")
    except Exception as e :
        print(f"Error deleting document {document.id} : {e}")

In [5]:
current_dir = os.path.curdir
file_path = os.path.join(current_dir, "video/messi-goals.mp4")
result = ragie.documents.create(request={
    "file": {
        "file_name":"messi-goals.mp4",
        "content": open(file_path, "rb"),
    },
    "mode": {
        "video": "audio_video"
    }
})

print(result)

status='partitioning' id='2e45bac0-967f-4e69-9573-69dfa2d37618' created_at=datetime.datetime(2025, 7, 31, 23, 48, 16, 662005, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 7, 31, 23, 48, 16, 710593, tzinfo=TzInfo(UTC)) name='messi-goals.mp4' metadata={} partition='default' chunk_count=None external_id=None page_count=None


In [6]:
response = ragie.retrievals.retrieve(request={
    "query": "Give detailed description of the video wit timestamp of the events"
})

print(response)

scored_chunks=[ScoredChunk(text='{"video_description": "Here\'s a detailed description of the video, focusing on the soccer plays:\\n\\n**Overall Impression:**\\n\\nThe video shows two significant moments from a soccer (football) match, likely a high-stakes game given the crowd and the intensity of the play. The first clip features a penalty kick. The second clip shows a close-up play near the goal, ending in a dramatic save.\\n\\n**Detailed Breakdown:**\\n\\n*   **Clip 1 (Penalty Kick):**\\n\\n    *   **Perspective:** The camera is positioned directly behind the goal, providing a clear view of the goalkeeper and the player taking the penalty.\\n    *   **Scene:** The stadium is packed with cheering fans, a palpable sense of tension in the air. Both teams are visible on the field. The score is 0-0 between Argentina and France.\\n    *   **Action:** The player approaches the ball, takes the shot, and scores the penalty. The goalkeeper dives to his left, but the ball is placed too well. 

In [7]:
content = []
for chunk in response.scored_chunks:
    print(chunk.text)
    print("#"*100)
    content.append({
        **chunk.document_metadata,
        "text": chunk.text,
        "document_name": chunk.document_name,
        "start_time": chunk.metadata.get("start_time"),
        "end_time": chunk.metadata.get("end_time")
    })

{"video_description": "Here's a detailed description of the video, focusing on the soccer plays:\n\n**Overall Impression:**\n\nThe video shows two significant moments from a soccer (football) match, likely a high-stakes game given the crowd and the intensity of the play. The first clip features a penalty kick. The second clip shows a close-up play near the goal, ending in a dramatic save.\n\n**Detailed Breakdown:**\n\n*   **Clip 1 (Penalty Kick):**\n\n    *   **Perspective:** The camera is positioned directly behind the goal, providing a clear view of the goalkeeper and the player taking the penalty.\n    *   **Scene:** The stadium is packed with cheering fans, a palpable sense of tension in the air. Both teams are visible on the field. The score is 0-0 between Argentina and France.\n    *   **Action:** The player approaches the ball, takes the shot, and scores the penalty. The goalkeeper dives to his left, but the ball is placed too well. The stadium erupts in celebration, with player

In [8]:
content[0]

{'text': '{"video_description": "Here\'s a detailed description of the video, focusing on the soccer plays:\\n\\n**Overall Impression:**\\n\\nThe video shows two significant moments from a soccer (football) match, likely a high-stakes game given the crowd and the intensity of the play. The first clip features a penalty kick. The second clip shows a close-up play near the goal, ending in a dramatic save.\\n\\n**Detailed Breakdown:**\\n\\n*   **Clip 1 (Penalty Kick):**\\n\\n    *   **Perspective:** The camera is positioned directly behind the goal, providing a clear view of the goalkeeper and the player taking the penalty.\\n    *   **Scene:** The stadium is packed with cheering fans, a palpable sense of tension in the air. Both teams are visible on the field. The score is 0-0 between Argentina and France.\\n    *   **Action:** The player approaches the ball, takes the shot, and scores the penalty. The goalkeeper dives to his left, but the ball is placed too well. The stadium erupts in c

In [ ]:
from moviepy import VideoFileClip

video = VideoFileClip("video/messi-goals.mp4")
goal_clip = video.subclipped(content[0]['start_time'], content[0]['end_time'])
goal_clip.write_videofile("video/goal_clip.mp4")

MoviePy - Building video video/goal_clip.mp4.
MoviePy - Writing audio in goal_clipTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video video/goal_clip.mp4



MoviePy - Done !
MoviePy - video ready video/goal_clip.mp4


In [12]:
from IPython.display import HTML
from pathlib import Path

# view the created video chunk
video_path = Path('video/goal_clip.mp4')
if video_path.exists():
    video_html= f'''
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    '''
    display(HTML(video_html))
else:
    print("Video nto found")
    